In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### install & import

In [ ]:
!pip3 install torch
!pip3 install torchvision
!pip3 install pytorch-lightning
!pip install pytorch_forecasting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 798 kB 5.2 MB/s 
     |████████████████████████████████| 512 kB 58.3 MB/s 
     |████████████████████████████████| 125 kB 75.5 MB/s 
     |████████████████████████████████| 87 kB 6.8 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=c9f50ab793e9f05d8afbf9adc6fe94fc15656021ff112fa201317972c50e8137
  Stored in directory: /root/.cache/pip/wheels/1f/10/06/2a990ee4d73a8479fe2922445e8a876d38cfbfed052284c6a1
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 141 kB 4.9 MB/s 
     |██████████████████████████████

In [ ]:
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch
import os

In [ ]:
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss, RMSE
#from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [ ]:
from pytorch_forecasting.metrics import MultiHorizonMetric
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

class TweedieLoss(MultiHorizonMetric):
    """
    Tweedie loss.

    Tweedie regression with log-link. It might be useful, e.g., for modeling total
    loss in insurance, or for any target that might be tweedie-distributed.

    The loss will take the exponential of the network output before it is returned as prediction.
    Target normalizer should therefore have no "reverse" transformation, e.g.
    for the :py:class:`~data.timeseries.TimeSeriesDataSet` initialization, one could use:

    .. code-block:: python

        from pytorch_forecasting import TimeSeriesDataSet, EncoderNormalizer

        dataset = TimeSeriesDataSet(
            target_normalizer=EncoderNormalizer(transformation=dict(forward=torch.log1p))
        )

    Note that in this example, the data is log1p-transformed before normalized but not re-transformed.
    The TweedieLoss applies this "exp"-re-transformation on the network output after it has been de-normalized.
    The result is the model prediction.
    """

    def __init__(self, reduction="mean", p: float = 1.5, **kwargs):
        """
        Args:
            p (float, optional): tweedie variance power which is greater equal
                1.0 and smaller 2.0. Close to ``2`` shifts to
                Gamma distribution and close to ``1`` shifts to Poisson distribution.
                Defaults to 1.5.
            reduction (str, optional): How to reduce the loss. Defaults to "mean".
        """
        super().__init__(reduction=reduction, **kwargs)
        assert 1 <= p < 2, "p must be in range [1, 2]"
        self.p = p

    def to_prediction(self, out: Dict[str, torch.Tensor]):
        rate = torch.exp(super().to_prediction(out))
        return rate


    def loss(self, y_pred, y_true):
        y_pred = super().to_prediction(y_pred)
        a = y_true * torch.exp(y_pred * (1 - self.p)) / (1 - self.p)
        b = torch.exp(y_pred * (2 - self.p)) / (2 - self.p)
        loss = -a + b
        return loss


### load data

In [ ]:
#os.chdir("/content/drive/MyDrive/소캡디/code")
os.chdir("/content/drive/MyDrive/4-1/소캡디/code")

##### training data (encode data) : 모델 load 용도로 사용

In [ ]:
data = pd.read_pickle("./data_use/CA_1_final_grid_df.pkl")

In [ ]:
data

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,sell_price,...,rolling_mean_60,rolling_std_60,rolling_mean_180,rolling_std_180,enc_cat_id_mean,enc_cat_id_std,enc_dept_id_mean,enc_dept_id_std,enc_item_id_mean,enc_item_id_std
0,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,1,12.0,0,0.459961,...,NaN,NaN,NaN,NaN,1.003906,3.115234,1.259766,3.533203,7.285156,9.179688
1,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,1,2.0,0,1.559570,...,NaN,NaN,NaN,NaN,1.003906,3.115234,1.259766,3.533203,1.178711,2.013672
2,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,0,3.169922,...,NaN,NaN,NaN,NaN,1.003906,3.115234,1.259766,3.533203,0.716797,0.919434
3,HOBBIES_1_012_CA_1_evaluation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,0,5.980469,...,NaN,NaN,NaN,NaN,1.003906,3.115234,1.259766,3.533203,0.392090,0.646973
4,HOBBIES_1_015_CA_1_evaluation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,1,4.0,0,0.700195,...,NaN,NaN,NaN,NaN,1.003906,3.115234,1.259766,3.533203,6.015625,7.324219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4873634,FOODS_3_823_CA_1_evaluation,FOODS_3_823,FOODS_3,FOODS,CA_1,CA,1969,NaN,127,2.980469,...,1.083008,1.356445,1.588867,1.838867,2.419922,5.882812,3.119141,7.214844,0.822266,1.382812
4873635,FOODS_3_824_CA_1_evaluation,FOODS_3_824,FOODS_3,FOODS,CA_1,CA,1969,NaN,0,2.480469,...,0.883301,1.462891,0.294434,0.937500,2.419922,5.882812,3.119141,7.214844,0.717773,1.205078
4873636,FOODS_3_825_CA_1_evaluation,FOODS_3_825,FOODS_3,FOODS,CA_1,CA,1969,NaN,1,3.980469,...,1.133789,1.016602,0.950195,1.115234,2.419922,5.882812,3.119141,7.214844,0.964844,1.354492
4873637,FOODS_3_826_CA_1_evaluation,FOODS_3_826,FOODS_3,FOODS,CA_1,CA,1969,NaN,211,1.280273,...,0.966797,1.301758,1.061523,1.415039,2.419922,5.882812,3.119141,7.214844,1.576172,1.843750


In [ ]:
values = {"rolling_mean_7": 0, "rolling_mean_14": 0, "rolling_mean_30": 0, "rolling_mean_60":0, "rolling_mean_180":0,
          "rolling_std_7":0,"rolling_std_14":0, "rolling_std_30":0, "rolling_std_60":0,"rolling_std_180":0}
data.fillna(values, inplace=True)
data.drop(columns = ['id','store_id','state_id'],inplace=True)

data["log_sales"] = np.log(data.sales + 0.0001)


In [ ]:
encode_data = data
#encode_data.drop(columns = ['id','store_id','state_id'],inplace=True)
#encode_data["sales"] = np.log(encode_data.sales + 1)
encode_data = encode_data.dropna(subset=['sales']) #test data 삭제
encode_data.reset_index(drop=True)

,item_id,dept_id,cat_id,d,sales,release,sell_price,price_max,price_min,price_std,...,rolling_std_60,rolling_mean_180,rolling_std_180,enc_cat_id_mean,enc_cat_id_std,enc_dept_id_mean,enc_dept_id_std,enc_item_id_mean,enc_item_id_std,log_sales
0,HOBBIES_1_008,HOBBIES_1,HOBBIES,1,12.0,0,0.459961,0.500000,0.419922,0.019760,...,0.000000,0.000000,0.000000,1.003906,3.115234,1.259766,3.533203,7.285156,9.179688,2.484915
1,HOBBIES_1_009,HOBBIES_1,HOBBIES,1,2.0,0,1.559570,1.769531,1.559570,0.032745,...,0.000000,0.000000,0.000000,1.003906,3.115234,1.259766,3.533203,1.178711,2.013672,0.693197
2,HOBBIES_1_010,HOBBIES_1,HOBBIES,1,0.0,0,3.169922,3.169922,2.970703,0.046356,...,0.000000,0.000000,0.000000,1.003906,3.115234,1.259766,3.533203,0.716797,0.919434,-9.210340
3,HOBBIES_1_012,HOBBIES_1,HOBBIES,1,0.0,0,5.980469,6.519531,5.980469,0.115967,...,0.000000,0.000000,0.000000,1.003906,3.115234,1.259766,3.533203,0.392090,0.646973,-9.210340
4,HOBBIES_1_015,HOBBIES_1,HOBBIES,1,4.0,0,0.700195,0.720215,0.680176,0.011337,...,0.000000,0.000000,0.000000,1.003906,3.115234,1.259766,3.533203,6.015625,7.324219,1.386319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4788262,FOODS_3_823,FOODS_3,FOODS,1941,2.0,127,2.980469,2.980469,2.480469,0.152222,...,1.242188,1.555664,1.825195,2.419922,5.882812,3.119141,7.214844,0.822266,1.382812,0.693197
4788263,FOODS_3_824,FOODS_3,FOODS,1941,0.0,0,2.480469,2.679688,2.470703,0.086365,...,1.166992,0.133301,0.696289,2.419922,5.882812,3.119141,7.214844,0.717773,1.205078,-9.210340
4788264,FOODS_3_825,FOODS_3,FOODS,1941,1.0,1,3.980469,4.378906,3.980469,0.189697,...,0.986328,0.850098,1.075195,2.419922,5.882812,3.119141,7.214844,0.964844,1.354492,0.000100
4788265,FOODS_3_826,FOODS_3,FOODS,1941,1.0,211,1.280273,1.280273,1.280273,0.000000,...,1.396484,1.016602,1.404297,2.419922,5.882812,3.119141,7.214844,1.576172,1.843750,0.000100


In [ ]:
encode_data

,item_id,dept_id,cat_id,d,sales,release,sell_price,price_max,price_min,price_std,...,rolling_std_60,rolling_mean_180,rolling_std_180,enc_cat_id_mean,enc_cat_id_std,enc_dept_id_mean,enc_dept_id_std,enc_item_id_mean,enc_item_id_std,log_sales
0,HOBBIES_1_008,HOBBIES_1,HOBBIES,1,12.0,0,0.459961,0.500000,0.419922,0.019760,...,0.000000,0.000000,0.000000,1.003906,3.115234,1.259766,3.533203,7.285156,9.179688,2.484915
1,HOBBIES_1_009,HOBBIES_1,HOBBIES,1,2.0,0,1.559570,1.769531,1.559570,0.032745,...,0.000000,0.000000,0.000000,1.003906,3.115234,1.259766,3.533203,1.178711,2.013672,0.693197
2,HOBBIES_1_010,HOBBIES_1,HOBBIES,1,0.0,0,3.169922,3.169922,2.970703,0.046356,...,0.000000,0.000000,0.000000,1.003906,3.115234,1.259766,3.533203,0.716797,0.919434,-9.210340
3,HOBBIES_1_012,HOBBIES_1,HOBBIES,1,0.0,0,5.980469,6.519531,5.980469,0.115967,...,0.000000,0.000000,0.000000,1.003906,3.115234,1.259766,3.533203,0.392090,0.646973,-9.210340
4,HOBBIES_1_015,HOBBIES_1,HOBBIES,1,4.0,0,0.700195,0.720215,0.680176,0.011337,...,0.000000,0.000000,0.000000,1.003906,3.115234,1.259766,3.533203,6.015625,7.324219,1.386319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4788262,FOODS_3_823,FOODS_3,FOODS,1941,2.0,127,2.980469,2.980469,2.480469,0.152222,...,1.242188,1.555664,1.825195,2.419922,5.882812,3.119141,7.214844,0.822266,1.382812,0.693197
4788263,FOODS_3_824,FOODS_3,FOODS,1941,0.0,0,2.480469,2.679688,2.470703,0.086365,...,1.166992,0.133301,0.696289,2.419922,5.882812,3.119141,7.214844,0.717773,1.205078,-9.210340
4788264,FOODS_3_825,FOODS_3,FOODS,1941,1.0,1,3.980469,4.378906,3.980469,0.189697,...,0.986328,0.850098,1.075195,2.419922,5.882812,3.119141,7.214844,0.964844,1.354492,0.000100
4788265,FOODS_3_826,FOODS_3,FOODS,1941,1.0,211,1.280273,1.280273,1.280273,0.000000,...,1.396484,1.016602,1.404297,2.419922,5.882812,3.119141,7.214844,1.576172,1.843750,0.000100


##### test data (new_prediction_data)

In [ ]:
##최근 90일 + 28일
new_prediction_data = data[lambda x: x.d > x.d.max() - 90 - 28]
new_prediction_data.reset_index(drop=True)
new_prediction_data.fillna({'sales': 0},inplace=True)
new_prediction_data.fillna({'log_sales': 0},inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
new_prediction_data

,item_id,dept_id,cat_id,d,sales,release,sell_price,price_max,price_min,price_std,...,rolling_std_60,rolling_mean_180,rolling_std_180,enc_cat_id_mean,enc_cat_id_std,enc_dept_id_mean,enc_dept_id_std,enc_item_id_mean,enc_item_id_std,log_sales
4513857,HOBBIES_1_001,HOBBIES_1,HOBBIES,1852,0.0,224,8.257812,9.578125,8.257812,0.152100,...,1.101562,0.683105,0.930176,1.003906,3.115234,1.259766,3.533203,0.605957,0.873047,-9.21034
4513858,HOBBIES_1_002,HOBBIES_1,HOBBIES,1852,0.0,20,3.970703,3.970703,3.970703,0.000000,...,1.065430,0.444336,0.860352,1.003906,3.115234,1.259766,3.533203,0.277588,0.585449,-9.21034
4513859,HOBBIES_1_003,HOBBIES_1,HOBBIES,1852,0.0,300,2.970703,2.970703,2.970703,0.000000,...,1.182617,0.672363,0.967773,1.003906,3.115234,1.259766,3.533203,0.366943,0.709473,-9.21034
4513860,HOBBIES_1_004,HOBBIES_1,HOBBIES,1852,1.0,5,4.640625,4.640625,4.339844,0.145264,...,1.683594,1.777344,2.009766,1.003906,3.115234,1.259766,3.533203,1.750977,1.991211,0.00010
4513861,HOBBIES_1_005,HOBBIES_1,HOBBIES,1852,0.0,16,2.880859,3.080078,2.480469,0.150146,...,1.241211,1.239258,1.229492,1.003906,3.115234,1.259766,3.533203,1.032227,1.308594,-9.21034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4873634,FOODS_3_823,FOODS_3,FOODS,1969,0.0,127,2.980469,2.980469,2.480469,0.152222,...,1.356445,1.588867,1.838867,2.419922,5.882812,3.119141,7.214844,0.822266,1.382812,0.00000
4873635,FOODS_3_824,FOODS_3,FOODS,1969,0.0,0,2.480469,2.679688,2.470703,0.086365,...,1.462891,0.294434,0.937500,2.419922,5.882812,3.119141,7.214844,0.717773,1.205078,0.00000
4873636,FOODS_3_825,FOODS_3,FOODS,1969,0.0,1,3.980469,4.378906,3.980469,0.189697,...,1.016602,0.950195,1.115234,2.419922,5.882812,3.119141,7.214844,0.964844,1.354492,0.00000
4873637,FOODS_3_826,FOODS_3,FOODS,1969,0.0,211,1.280273,1.280273,1.280273,0.000000,...,1.301758,1.061523,1.415039,2.419922,5.882812,3.119141,7.214844,1.576172,1.843750,0.00000


##### evaluation_data & weight

In [ ]:
eval_data = pd.read_pickle("./data_use/test_CA_1.pkl")

In [ ]:
eval_data

,item_id,dept_id,cat_id,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,2,0,1,0,0,1,4,...,2,1,2,0,0,1,0,1,3,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,0,2,0,1,0,1,0,...,1,0,0,1,0,0,2,1,1,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,0,0,0,0,0,1,0,...,1,3,2,1,0,2,1,0,1,1
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,0,0,1,0,6,3,3,...,3,3,4,2,1,6,3,1,4,3
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,2,0,1,1,2,4,0,...,0,1,2,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,FOODS_3_823,FOODS_3,FOODS,0,5,1,0,0,4,0,...,2,3,5,1,2,1,5,1,2,2
3045,FOODS_3_824,FOODS_3,FOODS,1,1,0,1,3,1,0,...,0,3,0,3,1,0,6,0,0,1
3046,FOODS_3_825,FOODS_3,FOODS,1,0,2,0,1,1,2,...,3,1,1,1,2,2,0,0,0,0
3047,FOODS_3_826,FOODS_3,FOODS,5,0,3,0,5,0,0,...,1,2,1,3,6,3,2,1,4,3


### load best model

In [ ]:
max_prediction_length = 28  # maximum prediction/decoder length (choose this not too short as it can help convergence) 
max_encoder_length = 90 # maximum length to encode. This is the maximum history length used by the time series dataset. #1년은 ram용량 없어짐 -> 로컬환경 구축
training_cutoff = data["d"].max() - max_prediction_length #training set에 사용할 time만큼

training = TimeSeriesDataSet(
    data[lambda x: x["d"] <= training_cutoff],
    time_idx="d",
    target="sales",
    group_ids=["item_id"],
    min_encoder_length= 1,
    max_encoder_length= max_encoder_length,
    min_prediction_length= 1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["item_id","dept_id","cat_id"], #시간이 지나도 변하지 않는 categorical
    static_reals=["release","price_max","price_min","price_std","price_mean","price_nunique",  "enc_cat_id_mean", "enc_cat_id_std", "enc_dept_id_mean", "enc_dept_id_std",
                  "enc_item_id_mean", "enc_item_id_std"], # 시간이 지나도 변하지 않는 continuous variables
    # list of categorical variables that change over time and are known in the future
    time_varying_known_categoricals=["snap_CA", "snap_TX","snap_WI","tm_w_end","event_name_1","event_name_2","event_type_1","event_type_2"], 
    # list of continuous variables that change over time and are known in the future
    time_varying_known_reals=["d","tm_d","tm_w",'tm_m',"tm_y","tm_wm","tm_dw","sell_price","price_norm","price_momentum","price_momentum_m","price_momentum_y",
                              "rolling_mean_7", "rolling_mean_14", "rolling_mean_30", "rolling_mean_60", "rolling_mean_180",
                              "rolling_std_7","rolling_std_14", "rolling_std_30", "rolling_std_60","rolling_std_180"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=["sales","log_sales"],
#    target_normalizer=GroupNormalizer(
#        groups=["item_id"], transformation="softplus"
#    ),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)


In [ ]:
loaded_model = tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.0845,
    hidden_size=20, 
    attention_head_size=1,
    dropout=0.265,
    hidden_continuous_size=10,
    output_size=1,  # point로 바꿔서 1로 변경
    loss= TweedieLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=4,
)

loaded_model.load_state_dict(torch.load('./model_build/tft_best_model_1207_store.pt'))
loaded_model.eval()

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


TemporalFusionTransformer(
  	"attention_head_size":               1
  	"categorical_groups":                {}
  	"causal_attention":                  True
  	"dropout":                           0.265
  	"embedding_labels":                  {'item_id': {'FOODS_1_001': 0, 'FOODS_1_002': 1, 'FOODS_1_003': 2, 'FOODS_1_004': 3, 'FOODS_1_005': 4, 'FOODS_1_006': 5, 'FOODS_1_008': 6, 'FOODS_1_009': 7, 'FOODS_1_010': 8, 'FOODS_1_011': 9, 'FOODS_1_012': 10, 'FOODS_1_013': 11, 'FOODS_1_014': 12, 'FOODS_1_015': 13, 'FOODS_1_016': 14, 'FOODS_1_017': 15, 'FOODS_1_018': 16, 'FOODS_1_019': 17, 'FOODS_1_020': 18, 'FOODS_1_021': 19, 'FOODS_1_022': 20, 'FOODS_1_023': 21, 'FOODS_1_024': 22, 'FOODS_1_025': 23, 'FOODS_1_026': 24, 'FOODS_1_027': 25, 'FOODS_1_028': 26, 'FOODS_1_029': 27, 'FOODS_1_030': 28, 'FOODS_1_031': 29, 'FOODS_1_032': 30, 'FOODS_1_033': 31, 'FOODS_1_034': 32, 'FOODS_1_035': 33, 'FOODS_1_036': 34, 'FOODS_1_037': 35, 'FOODS_1_038': 36, 'FOODS_1_039': 37, 'FOODS_1_040': 38, 'FOODS_1_041'

In [ ]:
del encode_data

### test data 예측

In [ ]:
pred_data = eval_data

In [ ]:
item_id_li = new_prediction_data['item_id'].unique()

In [ ]:
for item in item_id_li:
  pred_item = new_prediction_data.loc[new_prediction_data["item_id"]==item]
  pred_sales = loaded_model.predict(pred_item)
  for idx in range(28):
    pred_data.loc[(pred_data['item_id'] == item) ,'d_'+ str(1942+idx)] = float(pred_sales[0][idx])

In [ ]:
"""
for item in item_id_li:
  pred_item = new_prediction_data.loc[new_prediction_data["item_id"]==item]
  pred_sales = loaded_model.predict(pred_item)
  for idx in range(28):
    new_prediction_data.loc[(new_prediction_data['item_id'] == item) & (new_prediction_data['d'] == 1942+idx), 'sales'] = np.exp(float(pred_sales[0][idx]))
"""

'\nfor item in item_id_li:\n  pred_item = new_prediction_data.loc[new_prediction_data["item_id"]==item]\n  pred_sales = loaded_model.predict(pred_item)\n  for idx in range(28):\n    new_prediction_data.loc[(new_prediction_data[\'item_id\'] == item) & (new_prediction_data[\'d\'] == 1942+idx), \'sales\'] = np.exp(float(pred_sales[0][idx]))\n'

In [ ]:
pred_data

,item_id,dept_id,cat_id,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,0.795983,0.710614,0.693626,0.694853,0.791089,1.015329,1.154931,...,0.978780,1.219608,1.152204,0.868798,0.762882,0.752673,0.801738,0.948779,1.111888,1.091778
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,0.203983,0.238296,0.243894,0.243815,0.270096,0.333604,0.334803,...,0.272865,0.339193,0.359695,0.289571,0.270416,0.270282,0.272399,0.301665,0.363486,0.331568
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,0.447012,0.411903,0.414012,0.416742,0.471504,0.625803,0.628623,...,0.589803,0.686499,0.690258,0.495724,0.443328,0.449144,0.480942,0.587460,0.732258,0.682135
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,1.826839,1.578203,1.551172,1.574769,1.891437,2.261805,2.353551,...,1.896526,2.238258,2.069795,1.668405,1.416459,1.388610,1.445600,1.656069,2.052602,2.206143
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,1.132232,1.025386,1.028921,1.067891,1.209653,1.450123,1.471473,...,1.353666,1.584937,1.521357,1.195456,1.021569,0.994943,0.985774,1.153804,1.419311,1.382486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,FOODS_3_823,FOODS_3,FOODS,1.221946,1.080019,1.036878,1.045766,1.135973,1.529299,1.510492,...,1.310776,1.497076,1.535649,1.158801,0.993079,1.001022,1.090656,1.278233,1.701279,1.680872
3045,FOODS_3_824,FOODS_3,FOODS,0.655432,0.677963,0.733103,0.729303,0.827169,1.049366,1.060942,...,1.104938,1.270056,1.268971,0.943459,0.830872,0.817050,0.878677,1.011747,1.232358,1.203890
3046,FOODS_3_825,FOODS_3,FOODS,1.076785,0.917131,0.891824,0.898845,1.065902,1.272476,1.322542,...,1.204737,1.422538,1.357530,1.060888,0.890855,0.897086,0.900097,1.099966,1.352131,1.326903
3047,FOODS_3_826,FOODS_3,FOODS,1.449187,1.253233,1.235486,1.252934,1.456095,1.794157,1.703454,...,1.535600,1.846740,1.866344,1.475676,1.247992,1.273493,1.279332,1.499049,1.883448,1.848515


In [ ]:
pred_data.to_pickle('./data_use/pred_data_1207_store.pkl')

----------------------------------------------------------------------------------------------------------------------------------------------------------------